## 0. open + run the pre-trained model from Jaffeux 
(first time doing this)

containts code for messing around with inital opening + running of model

script also predict a single image

In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import csv

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable

# Define custom layer with serialization support
@register_keras_serializable()
class RandomFlip(tf.keras.layers.Layer):
    def __init__(self, mode='horizontal_and_vertical', seed=None, **kwargs):
        super(RandomFlip, self).__init__(**kwargs)
        self.mode = mode
        self.seed = seed

    def call(self, inputs, training=None):
        if training:
            if self.mode == 'horizontal_and_vertical':
                return tf.image.random_flip_left_right(tf.image.random_flip_up_down(inputs), seed=self.seed)
            elif self.mode == 'horizontal':
                return tf.image.random_flip_left_right(inputs, seed=self.seed)
            elif self.mode == 'vertical':
                return tf.image.random_flip_up_down(inputs, seed=self.seed)
        return inputs

    def get_config(self):
        config = super(RandomFlip, self).get_config()
        config.update({'mode': self.mode, 'seed': self.seed})
        return config

# Load the model with custom objects
custom_objects = {'RandomFlip': RandomFlip}
model = load_model('/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/novmodel.h5py', custom_objects=custom_objects)

# model = tf.keras.models.load_model('/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/DS_model/', custom_objects=custom_objects)


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 200, 200, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 198, 198, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 99, 99, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 128)       0         
 g2D)                                                   

In [21]:
### looking at a single image
# Load and preprocess your own data (example for a single image)
Jaffeux_data = '/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/DS_training_data/class_05_CG/' #class_01_Pri  class_02_Co  class_03_CC  class_04_ABC  class_05_CG  class_06_HC  class_07_Di  class_08_FA  class_09_WD
j_specif_image = '360524.png'

dcmex_2ds = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220730153000/'
specif_image = '969122_3.png'

#categories = ['Pri', 'Co', 'CC', 'ABC', 'CG', 'HC', 'Di', 'FA', 'WD'] # these are what training data contains - but don't match paper
categories = ['CA', 'Co',  'CC', 'CBC', 'CG', 'HPC', 'Dif', 'FA', 'WD'] # match paper predictions

#img_path = Jaffeux_data+j_specif_image
img_path = dcmex_2ds+specif_image

img = image.load_img(img_path, target_size=(200, 200), color_mode='grayscale')

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict using the loaded model
predictions = model.predict(img_array)[0]
print(predictions)

# Find the index of the category with the highest probability
predicted_index = np.argmax(predictions)

# Get the corresponding category
predicted_category = categories[predicted_index]

# Prepare the data to be saved
result = {'Name': specif_image[:-4], 'Category': predicted_category}
for i, category in enumerate(categories):
    result[category] = predictions[i]

# Save the predictions and the predicted category to a CSV file
with open('predictions_and_category.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name', 'Category'] + categories
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow(result)

1/1 [==============================] - 0s 69ms/step
[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
 6.2923784e-32 0.0000000e+00 0.0000000e+00 0.0000000e+00]


"\n# Save the predictions and the predicted category to a CSV file\nwith open('predictions_and_category.csv', 'w', newline='') as csvfile:\n    fieldnames = ['Name', 'Category']+ categories\n    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)\n\n    writer.writeheader()\n    writer.writerow({'Name': specif_image[:-4], 'Category': predicted_category})\n\nprint(f'The predicted category for {j_specif_image} is: {predicted_category}')\n"

The predicted category is: CG
